<a href="https://colab.research.google.com/github/Donalizasaji/LAB/blob/main/Dona_515_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample data: Replace with your parallel corpus (source, target pairs)
source_texts = ["hello", "how are you", "good morning"]
target_texts = ["hola", "cómo estás", "buenos días"]

# Hyperparameters
max_vocab_size = 10000
max_len = 10
embedding_dim = 256
units = 512
batch_size = 32
epochs = 10

# Tokenization and padding
source_tokenizer = Tokenizer(num_words=max_vocab_size, filters='')
target_tokenizer = Tokenizer(num_words=max_vocab_size, filters='')

source_tokenizer.fit_on_texts(source_texts)
target_tokenizer.fit_on_texts(target_texts)

source_sequences = source_tokenizer.texts_to_sequences(source_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)

source_padded = pad_sequences(source_sequences, maxlen=max_len, padding='post')
target_padded = pad_sequences(target_sequences, maxlen=max_len, padding='post')

source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

# Transformer model
class TransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(TransformerModel, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(units, return_sequences=True, return_state=True)
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, encoder_input, decoder_input):
        encoder_emb = self.embedding(encoder_input)
        encoder_output, encoder_state_h, encoder_state_c = self.lstm(encoder_emb)

        decoder_emb = self.embedding(decoder_input)
        decoder_output, _, _ = self.lstm(decoder_emb, initial_state=[encoder_state_h, encoder_state_c])

        output = self.dense(decoder_output)
        return output

# Instantiate model
model = TransformerModel(vocab_size=source_vocab_size, embedding_dim=embedding_dim, units=units)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (input: encoder_input, decoder_input, target_output)
encoder_input = source_padded
decoder_input = np.zeros_like(target_padded)
decoder_input[:, 1:] = target_padded[:, :-1]
target_output = np.expand_dims(target_padded, -1)

model.fit([encoder_input, decoder_input], target_output, batch_size=batch_size, epochs=epochs)

# Inference (translate new sentences)
def translate(input_sentence):
    input_seq = source_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')

    output_seq = np.zeros((1, max_len))
    for i in range(max_len):
        prediction = model.predict([input_seq, output_seq], verbose=0)
        predicted_token = np.argmax(prediction[0, i, :])
        output_seq[0, i] = predicted_token

        if predicted_token == target_tokenizer.word_index['<end>']:  # Use <end> token to stop early
            break

    translated_sentence = ' '.join([target_tokenizer.index_word[int(token)] for token in output_seq[0] if token != 0])
    return translated_sentence

# Test translation
translated_sentence = translate("hello")
print(f"Translated: {translated_sentence}")